### Lets Start with very simple SMA Strategy

In [1]:
import numpy
numpy.version.version

'1.26.4'

In [27]:
import datetime
import pandas_ta as ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

In [34]:
data_aapl = pd.read_csv('./dataset/AAPL/AAPL_1h.csv')
data_aapl.rename(columns={'timestamp': 'DatetimeIndex', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
data_aapl['DatetimeIndex'] = pd.to_datetime(data_aapl['DatetimeIndex'], unit='ms')
data_aapl.set_index('DatetimeIndex', inplace=True)
data_aapl

,Open,High,Low,Close,Volume
DatetimeIndex,,,,,
2010-01-04 09:00:00,7.5707,7.5886,7.5707,7.5886,8400.0
2010-01-04 10:00:00,7.5950,7.6000,7.5921,7.5921,11200.0
2010-01-04 11:00:00,7.5964,7.5964,7.5964,7.5964,2800.0
2010-01-04 12:00:00,7.5818,7.6204,7.5818,7.6071,562912.0
2010-01-04 13:00:00,7.6054,7.6104,7.5871,7.6071,5141136.0
...,...,...,...,...,...
2024-06-10 19:00:00,193.1600,194.1700,192.2900,193.1300,16383149.0
2024-06-10 20:00:00,193.1200,193.2100,192.2206,192.8500,1675871.0
2024-06-10 21:00:00,192.8916,193.1200,192.2200,192.3300,1055513.0


In [35]:
data_aapl.isnull().values.any()

False

In [45]:
data_aapl_D = data_aapl.resample('D', label='left').mean().interpolate(method='linear', limit_direction='forward')

In [42]:
data_aapl_D.isnull().values.any()

False

In [43]:
class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    # Do as much initial computation as possible
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)


    # Step through bars one by one
    # Note that multiple buys are a thing here

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

In [47]:
bt = Backtest(data_aapl_D, RsiOscillator, cash=10_000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()

Start                     2010-01-04 00:00:00
End                       2024-06-10 00:00:00
Duration                   5271 days 00:00:00
Exposure Time [%]                   37.575873
Equity Final [$]                 26876.835273
Equity Peak [$]                  29541.678672
Return [%]                         168.768353
Buy & Hold Return [%]             2455.994105
Return (Ann.) [%]                    7.084702
Volatility (Ann.) [%]               12.856027
Sharpe Ratio                          0.55108
Sortino Ratio                        0.831342
Calmar Ratio                         0.205793
Max. Drawdown [%]                  -34.426277
Avg. Drawdown [%]                   -4.061955
Max. Drawdown Duration     2041 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                                   43
Win Rate [%]                        79.069767
Best Trade [%]                      13.498299
Worst Trade [%]                    -28.123656
Avg. Trade [%]                    

/Users/satyam/Developer/Learn/algotrade/algo-tasks/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='3908', ...)

In [ ]:
bt = Backtest(data_aapl_D, RsiOscillator, cash=10_000, commission=.002)

stats = bt.optimize(
    upper_bound = range(55, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window = range(10, 30, 2)
)
stats = bt.run()
print(stats)
bt.plot()